In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
skf = StratifiedKFold(5, random_state = 123, shuffle = True)
for train_idx, valid_idx in skf.split(df_train[X_all], df_train['efs']):
    df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    t1, t2 = (
        df_cv_train.loc[df_cv_train['efs'] == 0.0, 'efs_time'].quantile(0.025),
        df_cv_train.loc[df_cv_train['efs'] == 1.0, 'efs_time'].quantile(0.6)
    )
    df_cv_train = df_cv_train.assign(efs_b = np.nan)
    df_cv_train.loc[(df_cv_train['efs'] == 0.0) & (df_cv_train['efs_time'] > t1), 'efs_b'] = 0
    df_cv_train.loc[(df_cv_train['efs'] == 1.0) & (df_train['efs_time'] < t2), 'efs_b'] = 1
    df_train_b = df_cv_train.loc[df_cv_train['efs_b'].notna()]
    print(t1, t2, len(df_train_b), len(df_train) - len(df_train_b))
    clf_lgb = lgb.LGBMClassifier(verbose = -1, n_estimators = 500, learning_rate = 0.02, class_weight={0.0: 1.0, 1.0: 1.0})
    clf_lgb.fit(df_train_b[X_all], df_train_b['efs_b'], categorical_feature = X_nom)
    prd1 = clf_lgb.predict_proba(df_train_b[X_all])[:, 1]
    prd2 = clf_lgb.predict_proba(df_cv_train[X_all])[:, 1]
    prd3 = clf_lgb.predict_proba(df_valid[X_all])[:, 1]
    print(
        roc_auc_score(df_train_b['efs'], prd1),
        roc_auc_score(df_cv_train['efs'], prd2),
        roc_auc_score(df_valid['efs'], prd3),
        concordance_index(df_train_b['efs_time'], -pd.Series(prd1, index = df_train_b.index), df_train_b['efs']),
        concordance_index(df_cv_train['efs_time'], -pd.Series(prd2, index = df_cv_train.index), df_cv_train['efs']),
        concordance_index(df_valid['efs_time'], -pd.Series(prd3, index = df_valid.index), df_valid['efs'])
    )